In [ ]:
# !pip install --upgrade --quiet pip
# !pip install --quiet torch==1.6.0 --index-url https://repo.usaa.com/artifactory/api/pypi/usaa-pypi-eval/simple --trusted-host repo.usaa.com
# !pip install --quiet /mnt/dgl_cu102-0.6.0-cp36-cp36m-manylinux1_x86_64.whl
# !pip install --upgrade --quiet tqdm

In [1]:
import argparse
import itertools
import os
import numpy as np
from numpy import save,load,savetxt,loadtxt,savez_compressed
from sklearn import metrics
from sklearn.metrics import roc_auc_score, f1_score,average_precision_score
from sklearn.metrics import precision_recall_fscore_support 
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc as auc_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import LabelEncoder, label_binarize

import pandas as pd
import scipy.sparse as sp
import time
from tqdm import tqdm, tqdm_notebook,tnrange
tqdm.pandas(position=0, leave=True)
import math 
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import dgl.function as Fn
from functools import partial
import seaborn as sns
import pickle
import random
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']=(5.0,4.0)
plt.rcParams['image.interpolation']='nearest'
plt.rcParams['image.cmap']='gray'
import warnings
warnings.filterwarnings('ignore')
import utils
import tsne_func
print("torch version is {}".format(th.__version__))
print("DGL version is {}".format(dgl.__version__))

Using backend: pytorch


torch version is 1.6.0
DGL version is 0.6.0


In [2]:
def seed_everything(seed):
    random.seed(seed)
    th.manual_seed(seed)
    th.cuda.manual_seed_all(seed)
    th.backends.cudnn.deterministic = True
    th.backends.cudnn.benchmark = False
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(101)

In [3]:
KG_dir="/home/ubuntu/"

if os.path.isfile("/home/ubuntu/CAP_Graph")==False:
    !hdfs dfs -get /dz/dz_6104/disc.db/CAP_Graph  ~/CAP_Graph

start=time.time()
with open(os.path.join(KG_dir,'CAP_Graph'), 'rb') as f:
    G, node_labels = pickle.load(f)
end=time.time()
print("It took {:0.4f} seconds to load graph".format(end-start))

ValueError: too many values to unpack (expected 2)

In [7]:
start=time.time()
with open(os.path.join(KG_dir,'CAP_Graph'), 'rb') as f:
    G,multi_label,binary_label,\
    train_mask_multi_label,  val_mask_multi_label,  test_mask_multi_label,\
    train_mask_binary_label, val_mask_binary_label, test_mask_binary_label= pickle.load(f)
end=time.time()
print("It took {:0.4f} seconds to load graph".format(end-start))

It took 5.4741 seconds to load graph


In [8]:
G

Graph(num_nodes={'usaanr': 25668504},
      num_edges={('usaanr', 'AUTO_RELATED', 'usaanr'): 7947060, ('usaanr', 'Brother_Sister', 'usaanr'): 651444, ('usaanr', 'Busi_rel_Other', 'usaanr'): 1039214, ('usaanr', 'Child', 'usaanr'): 11765795, ('usaanr', 'Ex-Spouse', 'usaanr'): 3877972, ('usaanr', 'Parent', 'usaanr'): 11765795, ('usaanr', 'Pers_rel_Other', 'usaanr'): 2012514, ('usaanr', 'SPONSEE', 'usaanr'): 16775141, ('usaanr', 'SPONSOR', 'usaanr'): 16775141, ('usaanr', 'Spouse', 'usaanr'): 16016119, ('usaanr', 'Step-Child', 'usaanr'): 1240193, ('usaanr', 'Step-Parent', 'usaanr'): 1240193},
      metagraph=[('usaanr', 'usaanr', 'AUTO_RELATED'), ('usaanr', 'usaanr', 'Brother_Sister'), ('usaanr', 'usaanr', 'Busi_rel_Other'), ('usaanr', 'usaanr', 'Child'), ('usaanr', 'usaanr', 'Ex-Spouse'), ('usaanr', 'usaanr', 'Parent'), ('usaanr', 'usaanr', 'Pers_rel_Other'), ('usaanr', 'usaanr', 'SPONSEE'), ('usaanr', 'usaanr', 'SPONSOR'), ('usaanr', 'usaanr', 'Spouse'), ('usaanr', 'usaanr', 'Step-Child')

In [9]:
utils.graph_show(G)

**************************************************
Node_types:  ['usaanr']
Edge_types:  ['AUTO_RELATED', 'Brother_Sister', 'Busi_rel_Other', 'Child', 'Ex-Spouse', 'Parent', 'Pers_rel_Other', 'SPONSEE', 'SPONSOR', 'Spouse', 'Step-Child', 'Step-Parent']
**************************************************
Canonical Etypes of Graph is:

usaanr              AUTO_RELATED        usaanr              
usaanr              Brother_Sister      usaanr              
usaanr              Busi_rel_Other      usaanr              
usaanr              Child               usaanr              
usaanr              Ex-Spouse           usaanr              
usaanr              Parent              usaanr              
usaanr              Pers_rel_Other      usaanr              
usaanr              SPONSEE             usaanr              
usaanr              SPONSOR             usaanr              
usaanr              Spouse              usaanr              
usaanr              Step-Child          usaanr          

In [ ]:
usaanr_feat=[]
for key, scheme in G.node_attr_schemes(ntype="usaanr").items():
    usaanr_feat.append(key)
usaanr_feat=[x for x in usaanr_feat if x not in ['ZIPCD','train_mask','val_mask','test_mask']]

print()
print("The features associated with USAA Member are\n ")
for i in usaanr_feat:
    print(i)

In [ ]:
## USAA Members Features Embedding
class USAANR_Embedding(nn.Module):
    def __init__(self,G,feature_size):
        super(USAANR_Embedding,self).__init__()
        self.G=G.to(device)
        self.feature_size=feature_size
        ## Embedding matrices for features of nodes.
        self.emb = nn.ModuleDict()
        
        for i,col in enumerate(usaanr_feat):
            self.emb[col]=nn.Embedding(G.nodes['usaanr'].data[col].max().item()+1, feature_size)
    
    def forward(self,nid):
        nid=nid.to(device)
        extra_repr=[]
        
        for i,col in enumerate(usaanr_feat):
            ndata=self.G.nodes['usaanr'].data[col]
            extra_repr.append(self.emb[col](ndata[nid]).squeeze(1))
        return th.stack(extra_repr, 0).sum(0)


In [ ]:
class RelGraphConvLayer(nn.Module):
    r"""Relational graph convolution layer.
    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    rel_names : list[str]
        Relation names.
    num_bases : int, optional
        Number of bases. If is none, use number of relations. Default: None.
    weight : bool, optional
        True if a linear layer is applied after message passing. Default: True
    bias : bool, optional
        True if bias is added. Default: True
    activation : callable, optional
        Activation function. Default: None
    self_loop : bool, optional
        True to include self loop message. Default: False
    dropout : float, optional
        Dropout rate. Default: 0.0
    """
    def __init__(self,
                 in_feat,
                 out_feat,
                 rel_names,
                 num_bases,
                 *,
                 weight=True,
                 bias=True,
                 activation=None,
                 self_loop=False,
                 dropout=0.0):
        super(RelGraphConvLayer, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.rel_names = rel_names
        self.num_bases = num_bases
        self.bias = bias
        self.activation = activation
        self.self_loop = self_loop
        self.conv = dglnn.HeteroGraphConv({
                rel : dglnn.GraphConv(in_feat, out_feat, norm="both", weight=False, bias=False)
                for rel in rel_names
            })
        self.use_weight = weight
        self.use_basis = num_bases < len(self.rel_names) and weight
        if self.use_weight:
            if self.use_basis:
                self.basis = dglnn.WeightBasis((in_feat, out_feat), num_bases, len(self.rel_names))
            else:
                self.weight = nn.Parameter(th.Tensor(len(self.rel_names), in_feat, out_feat))
                nn.init.xavier_uniform_(self.weight, gain=nn.init.calculate_gain('relu'))
        # bias
        if bias:
            self.h_bias = nn.Parameter(th.Tensor(out_feat))
            nn.init.zeros_(self.h_bias)
        # weight for self loop
        if self.self_loop:
            self.loop_weight = nn.Parameter(th.Tensor(in_feat, out_feat))
            nn.init.xavier_uniform_(self.loop_weight,
                                    gain=nn.init.calculate_gain('relu'))
        self.dropout = nn.Dropout(dropout)
    def forward(self, g, inputs):
        """Forward computation
        Parameters
        ----------
        g : DGLHeteroGraph
            Input graph.
        inputs : dict[str, torch.Tensor]
            Node feature for each node type.
        Returns
        -------
        dict[str, torch.Tensor]
            New node features for each node type.
        """
        g = g.local_var()
        if self.use_weight:
            weight = self.basis() if self.use_basis else self.weight
            wdict = {self.rel_names[i] : {'weight' : w.squeeze(0)}
                     for i, w in enumerate(th.split(weight, 1, dim=0))}
        else:
            wdict = {}
        if g.is_block:
            inputs_src = inputs
            inputs_dst = {k: v[:g.number_of_dst_nodes(k)] for k, v in inputs.items()}
        else:
            inputs_src = inputs_dst = inputs
        hs = self.conv(g, inputs, mod_kwargs=wdict)
        def _apply(ntype, h):
            if self.self_loop:
                h = h + th.matmul(inputs_dst[ntype], self.loop_weight)
            if self.bias:
                h = h + self.h_bias
            if self.activation:
                h = self.activation(h)
            return self.dropout(h)
        return {ntype : _apply(ntype, h) for ntype, h in hs.items()}

In [ ]:
class Entity_Classify(nn.Module):
    def __init__(self,
                 g,
                 h_dim,
                 out_dim,
                 num_bases,
#                  embed_layer,
                 num_hidden_layers=1,
                 dropout=0,
                 use_self_loop=False):
        super(Entity_Classify, self).__init__()
        self.g = g
        self.h_dim = h_dim
        self.out_dim = out_dim
        self.rel_names = list(set(g.etypes))
#         self.num_bases = None if num_bases < 0 else num_bases
        if num_bases < 0 or num_bases > len(self.rel_names):
            self.num_bases = len(self.rel_names)
        else:
            self.num_bases = num_bases
            
        self.num_hidden_layers = num_hidden_layers
        self.dropout = dropout
        self.use_self_loop = use_self_loop
        
#         self.node_embed={}
        self.node_embed=nn.ModuleDict()
        self.node_embed['usaanr'] = USAANR_Embedding(self.g,self.h_dim)
#         self.node_embed['zipcode'] = Zipcode_Embedding(self.g,self.h_dim)
        self.layers = nn.ModuleList()
        #i2h
        self.layers.append(RelGraphConvLayer(
                    self.h_dim, self.h_dim, self.rel_names,
                    self.num_bases, activation=F.relu, self_loop=self.use_self_loop,
                    dropout=self.dropout, weight=True))
        # h2h
        if self.num_hidden_layers>1:
            for i in range(0,self.num_hidden_layers-1):
                self.layers.append(RelGraphConvLayer(
                    self.h_dim, self.h_dim, self.rel_names,
                    self.num_bases, activation=F.relu, self_loop=self.use_self_loop,
                    dropout=self.dropout))
        # h2o
#         self.layers.append(RelGraphConvLayer(
#             self.h_dim, self.out_dim, self.rel_names, 
#             self.num_bases, activation=partial(F.softmax, dim=1),
#             self_loop=self.use_self_loop))
        self.classifier = nn.Linear(self.h_dim, self.out_dim)
    
    def forward(self, input_nodes, blocks=None):
        H={}
        for ntype, nid in input_nodes.items():
            nid = input_nodes[ntype]
            H[ntype] = self.node_embed[ntype](nid)
        if blocks is None:
            for layer in self.layers:
                H = layer(self.g, H)
        else:
            for layer, block in zip(self.layers, blocks):
                H = layer(block, H)
        output = self.classifier(H["usaanr"])
    
        return output, H["usaanr"]

In [ ]:
def get_class_count_weight(y,n_classes):
    classes_count=[]
    weight=[]
    for i in range(n_classes):
        count=np.sum(y.squeeze()==i)
        classes_count.append(count)
        weight.append(len(y)/(n_classes*count))
    return classes_count,weight

def eval_loop_func(model, loader, labels, device, loss_weight, num_classes):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    losses=[]
    for input_nodes_raw, seeds, blocks in tqdm(loader, position=0, leave=True):
        blocks = [blk.to(device) for blk in blocks]
        seeds = seeds.to(device)
        
        input_nodes={}
        input_nodes["usaanr"]=input_nodes_raw
        input_nodes={k : e.to(device) for k, e in input_nodes.items()}

        lbl = labels[seeds].squeeze().to(device)
        
        with th.no_grad():
            logits,h = model(input_nodes,blocks)
            if loss_weight is None:
                loss = F.cross_entropy(logits.view(-1, num_classes), lbl.to(device))
            else:
                loss = F.cross_entropy(logits.view(-1, num_classes), lbl.to(device),weight=th.Tensor(loss_weight).to(device))        
            losses.append(loss.item())
        fin_targets.append(lbl.cpu().detach().numpy())
        fin_outputs.append(logits.cpu().detach().numpy())
    return np.concatenate(fin_outputs), np.concatenate(fin_targets), losses

def evaluate(pred_test, y_test):
    
    ## convert logits into probability
    pred_test=th.nn.functional.softmax(th.from_numpy(pred_test),dim=1).numpy()
    
    acc = np.sum(pred_test.argmax(axis=1) == y_test.squeeze()) / y_test.shape[0]
    prec_macro, recall_macro, fscore_macro, _ = precision_recall_fscore_support(y_test.squeeze(), pred_test.argmax(axis=1), average='macro')
    prec_micro, recall_micro, fscore_micro, _ = precision_recall_fscore_support(y_test.squeeze(), pred_test.argmax(axis=1), average='micro')
    prec_weighted, recall_weighted, fscore_weighted, _ = precision_recall_fscore_support(y_test.squeeze(), pred_test.argmax(axis=1), average='weighted')
    
    macro_roc_auc_ovo=roc_auc_score(y_test,pred_test,multi_class="ovo",average="macro")
    weighted_roc_auc_ovo=roc_auc_score(y_test,pred_test,multi_class="ovo",average="weighted")

    macro_roc_auc_ovr=roc_auc_score(y_test,pred_test,multi_class="ovr",average="macro")
    weighted_roc_auc_ovr=roc_auc_score(y_test,pred_test,multi_class="ovr",average="weighted")
    
    
    _, count=np.unique(y_test,return_counts=True)
    weight=count/count.sum()
    
    y_test_binary=label_binarize(y_test, classes=np.unique(y_test).tolist())
    
    roc_auc = dict()
    pr_auc = dict()
    n_classes = y_test_binary.shape[1]
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_test_binary[:, i],pred_test[:, i])
        roc_auc[i] = auc_score(fpr, tpr)
        
        prec,rec,_ = precision_recall_curve(y_test_binary[:, i], th.sigmoid(th.from_numpy(pred_test))[:,i].numpy())
        pr_auc[i]=auc_score(rec,prec)

    # Compute micro-average ROC curve and ROC area
    fpr, tpr, _ = roc_curve(y_test_binary.ravel(), pred_test.ravel())
    roc_auc["micro"] = auc_score(fpr, tpr)
    roc_auc["macro"]=0
    roc_auc["weighted"]=0
    for i in range(n_classes):
        roc_auc["macro"]+=roc_auc[i]
        roc_auc["weighted"]+=roc_auc[i]*weight[i]
    roc_auc["macro"]/=n_classes
    
    prec,rec,_ = precision_recall_curve(y_test_binary.ravel(), th.sigmoid(th.from_numpy(pred_test)).numpy().ravel())
    pr_auc["micro"]=auc_score(rec,prec)

    pr_auc["macro"]=0
    pr_auc["weighted"]=0
    for i in range(n_classes):
        pr_auc["macro"]+=pr_auc[i]
        pr_auc["weighted"]+=pr_auc[i]*weight[i]
    pr_auc["macro"]/=n_classes

    metrics = {}
    metrics['acc'] = acc
    metrics['prec_macro'] = prec_macro
    metrics['recall_macro'] = recall_macro
    metrics['fscore_macro'] = fscore_macro

    metrics['prec_micro'] = prec_micro
    metrics['recall_micro'] = recall_micro
    metrics['fscore_micro'] = fscore_micro

    metrics['prec_weighted'] = prec_weighted
    metrics['recall_weighted'] = recall_weighted
    metrics['fscore_weighted'] = fscore_weighted
    
    metrics['auc_micro']=roc_auc["micro"]
    
    metrics['auc_macro_ovo']=macro_roc_auc_ovo
    metrics['auc_macro_ovr']=macro_roc_auc_ovr
    
    metrics['auc_weighted_ovo']=weighted_roc_auc_ovo
    metrics['auc_weighted_ovr']=weighted_roc_auc_ovr  
    
    metrics['pr_auc_micro']=pr_auc["micro"]
    metrics['pr_auc_macro']=pr_auc["macro"]
    metrics['pr_auc_weighted']=pr_auc["weighted"]

    return metrics, roc_auc, pr_auc


In [ ]:
# def evaluate(model, loader, labels, category, device):
#     model.eval()
#     total_loss = 0
#     total_acc = 0
# #     total_precision=0
# #     total_recall=0
#     total_fscore_macro=0
#     total_fscore_micro=0
#     total_fscore_weighted=0
# #     total_auc=0
# #     total_pr_auc=0
    
#     count = 0
#     count_loss=0
    
#     y_pred=[]
#     y_true=[]
    
#     for input_nodes_raw, seeds_raw, blocks in tqdm(loader, position=0, leave=True):
#         blocks = [blk.to(device) for blk in blocks]
        
#         seeds=seeds_raw.to(device)
        
#         input_nodes={}
#         input_nodes[category]=input_nodes_raw
#         input_nodes={k : e.to(device) for k, e in input_nodes.items()}
        
#         lbl = labels[seeds].to(device)
#         logits,h = model(input_nodes,blocks)
#         loss = F.cross_entropy(logits, lbl.squeeze(1).to(device))
# #         loss = F.cross_entropy(logits, lbl.squeeze(1),weight=th.Tensor([1,args.weight]).to(device))
#         acc = th.sum(logits.argmax(dim=1) == lbl.squeeze(1)).item() / logits.shape[0]
# #         precision, recall, fscore, support = score(lbl.squeeze(1).cpu().numpy(), logits.argmax(dim=1).cpu().numpy())

#         precision_macro, recall_macro, fscore_macro, _ = score(lbl.squeeze().cpu().numpy(), logits.argmax(dim=1).cpu().numpy(), average='macro')
#         precision_micro, recall_micro, fscore_micro, _ = score(lbl.squeeze().cpu().numpy(), logits.argmax(dim=1).cpu().numpy(), average='micro')
#         precision_weighted, recall_weighted, fscore_weighted, _ = score(lbl.squeeze().cpu().numpy(), logits.argmax(dim=1).cpu().numpy(), average='weighted')
        
#         y_pred.extend(logits.argmax(dim=1).tolist())
#         y_true.extend(lbl.squeeze().tolist())
        
# #         tempt=lbl.detach().cpu().numpy()
# #         labels_train_one_hot=np.zeros(shape=(tempt.shape[0],8),dtype=np.float32)
# #         labels_train_one_hot[np.arange(tempt.shape[0]),np.array([ele.item() for ele in tempt])]=1
# #         auc = roc_auc_score(labels_train_one_hot.ravel(), th.sigmoid(logits).detach().cpu().numpy().ravel())
        
# #         auc = roc_auc_score(lbl.detach().cpu().numpy().ravel(), th.sigmoid(logits)[:,1].detach().cpu().numpy().ravel())
# #         prec,rec,_ = precision_recall_curve(lbl.detach().cpu().numpy().ravel(), th.sigmoid(logits)[:,1].detach().cpu().numpy().ravel())
# #         pr_auc=auc_score(rec,prec)
        
#         total_loss += loss.item() * len(seeds) 
#         total_acc += acc
# #         total_precision += precision[1]
# #         total_recall += recall[1]
#         total_fscore_macro += fscore_macro
#         total_fscore_micro += fscore_micro
#         total_fscore_weighted += fscore_weighted
# #         total_auc += auc
# #         total_pr_auc += pr_auc
#         count += 1
#         count_loss += len(seeds)
    
#     ACCURACY=total_acc / count
#     LOSS=total_loss / count_loss
# #     PRECISION=total_precision/count
# #     RECALL=total_recall/count
#     F1_SCORE_macro=total_fscore_macro/count
#     F1_SCORE_micro=total_fscore_micro/count
#     F1_SCORE_weighted=total_fscore_weighted/count
# #     AUC=total_auc/count
# #     PR_AUC=total_pr_auc/count
    
#     return ACCURACY, LOSS, F1_SCORE_macro, F1_SCORE_micro, F1_SCORE_weighted, y_pred, y_true

In [ ]:
def inference(model, graph, input_features, batch_size):
    """Minibatch inference of final representation over all node types.
    ***NOTE***
    For node classification, the model is trained to predict on only one node type's
    label.  Therefore, only that type's final representation is meaningful.
    """
    with th.no_grad():
        for l, layer in enumerate(model.layers):
            
            output_features = {
                k: th.zeros(
                    graph.number_of_nodes(k),
                    args.h_dim )
                for k in graph.ntypes}
            sampler = dgl.dataloading.MultiLayerFullNeighborSampler(1)
            dataloader = dgl.dataloading.NodeDataLoader(
                graph,
                {k: th.arange(graph.number_of_nodes(k)) for k in graph.ntypes},
                sampler,
                batch_size=batch_size,
                shuffle=True,
                drop_last=False,
                num_workers=0)
            
#             dataloader=[next(iter(dataloader)) for idx in range(10)]
            for input_nodes, output_nodes, blocks in tqdm(dataloader, position=0, leave=True):
                block = blocks[0].to(device)
                h = {k: input_features[k][input_nodes[k]].to(device) for k in input_nodes.keys()}
                h = layer(block, h)
                for k in h.keys():
#                     h[k]=th.matmul(h[k],model.classifier.weight.t())+model.classifier.bias
                    output_features[k][output_nodes[k]] = h[k].cpu()
            input_features = output_features
        
    return model.classifier(output_features["usaanr"])

In [ ]:
# dict_edges={}
# for etype in G.etypes:
#     dict_edges[etype]=th.arange(G.num_edges(etype))[0:5000]
# G=dgl.edge_subgraph(G,dict_edges)

# G.nodes['usaanr'].data["_ID"].numpy().shape,node_labels.shape, node_labels[G.nodes['usaanr'].data["_ID"]].shape

#### settings

In [ ]:
parser = argparse.ArgumentParser(description='RGCN')
parser.add_argument("--dropout", type=float, default=0,
        help="dropout probability")
parser.add_argument("--h_dim", type=int, default=128,
        help="number of hidden units")
parser.add_argument("--out_dim", type=int, default=1,
        help="output dimension")
parser.add_argument("--gpu", type=int, default=0,
        help="gpu")
parser.add_argument("--lr", type=float, default=1e-5,
        help="learning rate")
parser.add_argument("--num_bases", type=int, default=-1,
        help="number of filter weight matrices, default: -1 [use all]")
parser.add_argument("--num_layers", type=int, default=1,
        help="number of propagation rounds")
parser.add_argument("-e", "--n_epochs", type=int, default=1,
        help="number of training epochs")
parser.add_argument("--model_path", type=str, default="/workspace/cjiang/eagle_project/CAP_graph/CAP_without_zipcode/rgcn_model_param.pt",
        help='path for save the model')
parser.add_argument("--l2norm", type=float, default=0,
        help="l2 norm coef")
parser.add_argument("--use_self_loop", default=True, action='store_true',
        help="include self feature as a special relation")
parser.add_argument("--batch-size", type=int, default=1024,
        help="Mini-batch size. If -1, use full graph training.")
parser.add_argument("--num_mini_batch", type=int, default=8,
        help="Number of minibatch.")
parser.add_argument("--fanout", type=int, default=None,
        help="Fan-out of neighbor sampling.")
parser.add_argument("--validation",  default=True,
        help="set up validation .")
parser.add_argument("--seed",  type=int,default=101,
        help="random seed for np.random.seed, torch.manual_seed and torch.cuda.manual_seed.")

parser.add_argument("--weight",  type=float,default=1,  ## number of label=0/number of label=1
        help="weight for unbalance data")
parser.add_argument("--num_worker",  type=int,default=0,  
        help="number of worker for neighbor sampling") 
    
args,unknown=parser.parse_known_args()

In [ ]:
args.num_layers=1
args.dropout=0.2
args.lr=1e-3
args.l2norm=1e-3
args.n_epochs=1
args.num_bases=5
args.h_dim=64
args.batch_size=1024*10
print(args)

#### regroup label to eliminate rare category

In [ ]:
y=node_labels.squeeze().numpy()
n_classes=th.unique(node_labels).shape[0]
classes_count,weight=get_class_count_weight(y,n_classes)
imbalance_classes={}
imbalance_classes["category"]=th.unique(node_labels).tolist()
imbalance_classes["product_type"]=["No Product","Rental Only","Home Only","Home + Rental","Auto Only","Auto + Rental","Auto + Home","Auto + Home + Rental"]
imbalance_classes["count"]=classes_count
imbalance_classes["weight"]=weight
imbalance_classes=pd.DataFrame(imbalance_classes)
imbalance_classes.style.format({"count":"{:,}","weight":"{:.2f}"})

In [ ]:
node_labels_new=node_labels.clone()
node_labels_new[node_labels_new==3]=2
node_labels_new[node_labels_new==4]=3
node_labels_new[node_labels_new==5]=4
node_labels_new[node_labels_new==6]=5
node_labels_new[node_labels_new==7]=5

label_map={0:"No Product",1:"Rental Only",2:"Home Only",3:"Auto Only",4:"Auto + Rental",5:"Auto + Home"}

y=node_labels_new.squeeze().numpy()
n_classes=th.unique(node_labels_new).shape[0]
classes_count,weight=get_class_count_weight(y,n_classes)
imbalance_classes={}
imbalance_classes["category"]=th.unique(node_labels_new).tolist()
# imbalance_classes["product_type"]=["No Product","Rental Only","Home Only","Auto Only","Auto + Rental","Auto + Home"]
imbalance_classes["product_type"]=[label_map[i] for i in range(n_classes)]
imbalance_classes["count"]=classes_count
imbalance_classes["weight"]=weight
imbalance_classes=pd.DataFrame(imbalance_classes)
imbalance_classes.style.format({"count":"{:,}","weight":"{:.2f}"})

In [ ]:
import seaborn as sns
sns.distplot(node_labels_new)

#### setting up training, validation and test set

In [ ]:
train_mask=G.nodes["usaanr"].data.pop('train_mask')
val_mask=G.nodes["usaanr"].data.pop('val_mask')
test_mask=G.nodes["usaanr"].data.pop('test_mask')

mask=((train_mask==True) | (val_mask==True))

train_idx=th.nonzero(train_mask.squeeze(1)).numpy()
val_idx=th.nonzero(val_mask.squeeze(1)).numpy()
test_idx=th.nonzero(test_mask.squeeze(1)).numpy()

train_idx=th.from_numpy(train_idx).squeeze(1)    
val_idx=th.from_numpy(val_idx).squeeze(1)    
test_idx=th.from_numpy(test_idx).squeeze(1)

train_label=node_labels_new[train_idx].squeeze().numpy()
val_label=node_labels_new[val_idx].squeeze().numpy()
test_label=node_labels_new[test_idx].squeeze().numpy()

print('{:<15} {:<10,}'.format("Training set",train_idx.shape[0]))
print('{:<15} {:<10,}'.format("validation set",val_idx.shape[0]))
print('{:<15} {:<10,}'.format("test set",test_idx.shape[0]))


In [ ]:
##### check cuda
device="cpu"
use_cuda=args.gpu>=0 and th.cuda.is_available()
if use_cuda:
    th.cuda.set_device(args.gpu)
    device='cuda:%d' % args.gpu
print(device)

In [ ]:
# create model
model = Entity_Classify(G,
                       args.h_dim,
                       n_classes,
                       num_bases=args.num_bases,
                       num_hidden_layers=args.num_layers,
                       dropout=args.dropout,
                       use_self_loop=args.use_self_loop)
if use_cuda:
    model.cuda()

model.layers

In [ ]:
optimizer = th.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2norm)

In [ ]:
# train sampler
sampler = dgl.dataloading.MultiLayerNeighborSampler([args.fanout] * args.num_layers)
train_loader = dgl.dataloading.NodeDataLoader(
    G, {'usaanr': train_idx}, sampler,
    batch_size=args.batch_size, shuffle=True, num_workers=args.num_worker)
# validation sampler
# we do not use full neighbor to save computation resources
val_sampler = dgl.dataloading.MultiLayerNeighborSampler([args.fanout] * args.num_layers)
val_loader = dgl.dataloading.NodeDataLoader(
    G, {'usaanr': val_idx}, val_sampler,
    batch_size=args.batch_size, shuffle=False, num_workers=args.num_worker)

test_sampler = dgl.dataloading.MultiLayerNeighborSampler([args.fanout] * args.num_layers)
test_loader = dgl.dataloading.NodeDataLoader(
    G, {'usaanr': test_idx}, test_sampler,
    batch_size=args.batch_size, shuffle=False, num_workers=args.num_worker)

print("The number of minibatch in training set is {:,}".format(len(train_loader)))
print("The number of minibatch in validation set is {:,}".format(len(val_loader)))
print("The number of minibatch in test set is {:,}".format(len(test_loader)))

In [ ]:
print("The total # of parameter is {:,}".format(sum([p.nelement() for p in model.parameters()]) ) )

In [ ]:
param_dict={n: p.nelement() for n, p in model.named_parameters()}
for i,j in param_dict.items():
    print("{:<70}{:<15,}".format(i,j))

In [ ]:
# %pdb
LOSS_EPOCH=[]
LABEL_TRAIN=[]

# training loop
print("start training...")

total_loss=0
losses=[]

train_true=[]
train_pred=[]

# th.manual_seed(args.seed)
# th.cuda.manual_seed(args.seed)
# th.cuda.manual_seed_all(args.seed)
# np.ranom.seed(args.seed)
# random.seed(args.seed)
# th.backends.cudnn.deterministic=True

for epoch in tqdm(range(0,args.n_epochs)):
    
    model.train()
    IDX=[]
    H=[]
    
    #====================================#
    #            Traning                 #
    #====================================#
    print("")
    print("========= Epoch {:} /{:}".format(epoch+1,args.n_epochs))
    print("Training...")
    t0 = time.time()
    for step, (input_nodes_raw, seeds_raw, blocks) in enumerate(train_loader):
        blocks = [blk.to(device) for blk in blocks]
        
        seeds=seeds_raw.to(device)
        
        labels_train=node_labels_new[seeds]       
        labels_train = labels_train.to(device)
        
        input_nodes={}
        input_nodes["usaanr"]=input_nodes_raw
        input_nodes={k : e.to(device) for k, e in input_nodes.items()}
        
        logits,h = model(input_nodes,blocks)
        optimizer.zero_grad()
        loss = F.cross_entropy(logits, labels_train.squeeze(1),weight=th.Tensor(weight).to(device))
#         loss = F.cross_entropy(logits, labels_train.squeeze(1).to(device))
        total_loss+=loss.item()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
        train_pred.extend(logits.argmax(dim=1).tolist())
        train_true.extend(labels_train.squeeze(1).tolist())

        train_acc = th.sum(logits.argmax(dim=1) == labels_train.squeeze(1)).item() / len(seeds)
        precision_macro, recall_macro, fscore_macro, _ = precision_recall_fscore_support(labels_train.squeeze(1).cpu().numpy(), logits.argmax(dim=1).cpu().numpy(), average='macro')
        precision_micro, recall_micro, fscore_micro, _ = precision_recall_fscore_support(labels_train.squeeze(1).cpu().numpy(), logits.argmax(dim=1).cpu().numpy(), average='micro')
        precision_weighted, recall_weighted, fscore_weighted, _ = precision_recall_fscore_support(labels_train.squeeze(1).cpu().numpy(), logits.argmax(dim=1).cpu().numpy(), average='weighted')

#         macro_roc_auc_ovo=roc_auc_score(labels_train,logits,multi_class="ovo",average="macro")
#         weighted_roc_auc_ovo=roc_auc_score(labels_train,logits,multi_class="ovo",average="weighted")
        
        
#         _, count=np.unique(labels_train,return_counts=True)
#         W=count/count.sum()

#         labels_train_binary=label_binarize(labels_train, classes=np.unique(labels_train).tolist())

#         roc_auc = dict()
#         pr_auc = dict()
#         n_classes = labels_train_binary.shape[1]
#         for i in range(n_classes):
#             fpr, tpr, _ = roc_curve(labels_train_binary[:, i],preds[:, i])
#             roc_auc[i] = auc_score(fpr, tpr)

#             prec,rec,_ = precision_recall_curve(labels_train_binary[:, i], th.sigmoid(th.from_numpy(logits))[:,i].numpy())
#             pr_auc[i]=auc_score(rec,prec)

#         # Compute micro-average ROC curve and ROC area
#         fpr, tpr, _ = roc_curve(labels_train_binary.ravel(), logits.ravel())
#         roc_auc["micro"] = auc_score(fpr, tpr)
#         roc_auc["macro"]=0
#         roc_auc["weighted"]=0
#         for i in range(n_classes):
#             roc_auc["macro"]+=roc_auc[i]
#             roc_auc["weighted"]+=roc_auc[i]*weight[i]
#         roc_auc["macro"]/=n_classes

#         prec,rec,_ = precision_recall_curve(labels_train_binary.ravel(), torch.sigmoid(torch.from_numpy(logits)).numpy().ravel())
#         pr_auc["micro"]=auc_score(rec,prec)

#         pr_auc["macro"]=0
#         pr_auc["weighted"]=0
#         for i in range(n_classes):
#             pr_auc["macro"]+=pr_auc[i]
#             pr_auc["weighted"]+=pr_auc[i]*weight[i]
#         pr_auc["macro"]/=n_classes


        IDX.extend(seeds.detach().cpu().numpy().tolist())
        H.extend(h.detach().cpu().numpy().tolist())
        
        if step%(len(train_loader)//10)==0 and not step==0:

            t1 = time.time()
            elapsed=utils.format_time(t1-t0)
            print("Batch {:} of {:} | Loss {:.3f}  | Accuracy {:.2%} | F1-Score(macro) {:.2%}  | F1-Score(micro) {:.2%} | F1-Score(weighted) {:.2%} | Elapsed: {:}".\
                  format(step,len(train_loader),np.mean(losses[-10:]),train_acc,fscore_macro,fscore_micro,fscore_weighted,elapsed))    
    
    LOSS_EPOCH.append(loss)

    LABEL_TRAIN.append(node_labels_new[blocks[-1].nodes["usaanr"].data[dgl.NID].cpu().numpy()])


    model.eval()
    print()
    print("")
    print("Running Validation on training set")
    print("")
    fin_outputs, fin_targets, losses_tmp=eval_loop_func(model, train_loader, node_labels_new,  device, weight, n_classes)
    
    avg_loss_train=np.mean(losses_tmp)
    
    metrics_train, roc_auc_train, pr_auc_train = evaluate(fin_outputs, fin_targets)
    
    t2=time.time()
    
    print("loss: {:.3f} |  Accuracy {:.2%} | F1-Score(macro) {:.2%}  | F1-Score(weighted) {:.2%}  | roc-auc(macro) {:.2%}  | roc-auc(weighted) {:.2%}  | pr-auc(macro) {:.2%}  | pr-auc(weighted) {:.2%}  | Elapsed: {:}"\
      .format(avg_loss_train, metrics_train["acc"],  metrics_train['fscore_macro'], metrics_train['fscore_weighted'],  \
              metrics_train['auc_macro_ovo'], metrics_train['auc_weighted_ovo'],metrics_train['pr_auc_macro'], \
              metrics_train['pr_auc_weighted'],utils.format_time(t2-t1)))

    #====================================#
    #            Validation-set          #
    #====================================#
    
    model.eval()
    print()
    print("")
    print("Running Validation on validation set")
    print("")
    
    fin_outputs, fin_targets, losses_tmp=eval_loop_func(model, val_loader, node_labels_new, device, weight, n_classes)
    
    avg_loss_val=np.mean(losses_tmp)
    
    metrics_val, roc_auc_val, pr_auc_val = evaluate(fin_outputs, fin_targets)
    
    t3=time.time()
    
    print("loss: {:.3f} |  Accuracy {:.2%} | F1-Score(macro) {:.2%}  | F1-Score(weighted) {:.2%}  | roc-auc(macro) {:.2%}  | roc-auc(weighted) {:.2%}  | pr-auc(macro) {:.2%}  | pr-auc(weighted) {:.2%}  | Elapsed: {:}"\
          .format(avg_loss_val, metrics_val["acc"],  metrics_val['fscore_macro'], metrics_val['fscore_weighted'],  \
          metrics_val['auc_macro_ovo'], metrics_val['auc_weighted_ovo'],metrics_val['pr_auc_macro'], \
          metrics_val['pr_auc_weighted'],utils.format_time(t3-t2)))
    
# if args.model_path is not None:
#     th.save(model.state_dict(), args.model_path)
    
#====================================#
#            Test-set                #
#====================================#
print()
print("")
print("Running Validation in Test Dataset")
print("")
model.eval()

fin_outputs, fin_targets, losses_tmp=eval_loop_func(model, test_loader, node_labels_new,  device, weight, n_classes)
    
avg_loss_test=np.mean(losses_tmp)

metrics_test, roc_auc_test, pr_auc_test = evaluate(fin_outputs, fin_targets)

t4=time.time()

print("loss: {:.3f} |  Accuracy {:.2%} | F1-Score(macro) {:.2%}  | F1-Score(weighted) {:.2%}  | roc-auc(macro) {:.2%}  | roc-auc(weighted) {:.2%}  | pr-auc(macro) {:.2%}  | pr-auc(weighted) {:.2%}  | Elapsed: {:}"\
  .format(avg_loss_test, metrics_test["acc"],  metrics_test['fscore_macro'], metrics_test['fscore_weighted'],  \
          metrics_test['auc_macro_ovo'], metrics_test['auc_weighted_ovo'],metrics_test['pr_auc_macro'], \
          metrics_test['pr_auc_weighted'],utils.format_time(t4-t3)))


In [ ]:
%%time
print()
print("********  model performance in training set **********  \n ")
print()
    
fin_outputs, fin_targets, losses_tmp=eval_loop_func(model, train_loader, node_labels_new, device, weight, n_classes)
metrics_dict, roc_auc, pr_auc = evaluate(fin_outputs, fin_targets)

print("{:<20}{:<10.2%}".format("accuracy", metrics_dict['acc']))
print()
print("{:<20}{:<10,.2%}{:<16}{:<10,.2%}{:<18}{:<10,.2%}{:<17}{:<10,.2%}{:<16}{:<10,.2%}"\
      .format("precision(macro):",metrics_dict['prec_macro'],"recall(macro):",metrics_dict['recall_macro'],\
              "f1-score(macro):",metrics_dict['fscore_macro'],"ROC-AUC(macro):",metrics_dict['auc_macro_ovo'],\
             "PR-AUC(macro):",metrics_dict['pr_auc_macro']))

print("{:<20}{:<10,.2%}{:<16}{:<10,.2%}{:<18}{:<10,.2%}{:<17}{:<10,.2%}{:<16}{:<10,.2%}"\
      .format("precision(micro):",metrics_dict['prec_micro'],"recall(micro):",metrics_dict['recall_micro'],\
              "f1-score(micro):",metrics_dict['fscore_micro'],"ROC-AUC(micro):",metrics_dict['auc_micro'],\
             "PR-AUC(micro):",metrics_dict['pr_auc_micro']))

print("{:<20}{:<10,.2%}{:<16}{:<10,.2%}{:<18}{:<10,.2%}{:<17}{:<10,.2%}{:<16}{:<10,.2%}"\
      .format("precision(weight):",metrics_dict['prec_weighted'],"recall(weight):",metrics_dict['recall_weighted'],\
              "f1-score(weight):",metrics_dict['fscore_weighted'],"ROC-AUC(weight):",metrics_dict['auc_weighted_ovo'],\
             "PR-AUC(weight):",metrics_dict['pr_auc_weighted']))

print()
n_classes=th.unique(node_labels_new).shape[0]
label_map={0:"No Product",1:"Rental Only",2:"Home Only",3:"Auto Only",4:"Auto + Rental",5:"Auto + Home"}

report=metrics.classification_report(fin_targets.squeeze(), fin_outputs.argmax(axis=1), output_dict=True)

table = pd.DataFrame(report).transpose().iloc[:n_classes,:]
table["count"]=table["support"].astype(int)
table["roc_auc"]=[roc_auc[i] for i in range(n_classes)]
table["pr_auc"]=[pr_auc[i] for i in range(n_classes)]
table["product_type"]=[label_map[i] for i in range(n_classes)]
table=table[['product_type','count','precision','recall','f1-score','roc_auc','pr_auc']]

total=table['count'].sum()

table.loc[len(table.index)]=["MACRO",total,metrics_dict['prec_macro'],metrics_dict['recall_macro'],metrics_dict['fscore_macro'],\
                        metrics_dict['auc_macro_ovo'],metrics_dict['pr_auc_macro']]

table.loc[len(table.index)]=["MICRO",total,metrics_dict['prec_micro'],metrics_dict['recall_micro'],metrics_dict['fscore_micro'],\
                            metrics_dict['auc_micro'],metrics_dict['pr_auc_micro']]

table.loc[len(table.index)]=["WEIGHT",total,metrics_dict['prec_weighted'],metrics_dict['recall_weighted'],metrics_dict['fscore_weighted'],\
                        metrics_dict['auc_weighted_ovo'],metrics_dict['pr_auc_weighted']]

table.style.format({"count":"{:,}","f1-score":"{:.2%}","precision":"{:.2%}","recall":"{:.2%}","roc_auc":"{:.2%}","pr_auc":"{:.2%}"})

In [ ]:
%%time
print()
print("********  model performance in validation set **********  \n ")
print()
    
fin_outputs, fin_targets, losses_tmp=eval_loop_func(model, val_loader, node_labels_new, device, weight, n_classes)
metrics_dict, roc_auc, pr_auc = evaluate(fin_outputs, fin_targets)

print("{:<20}{:<10.2%}".format("accuracy", metrics_dict['acc']))
print()
print("{:<20}{:<10,.2%}{:<16}{:<10,.2%}{:<18}{:<10,.2%}{:<17}{:<10,.2%}{:<16}{:<10,.2%}"\
      .format("precision(macro):",metrics_dict['prec_macro'],"recall(macro):",metrics_dict['recall_macro'],\
              "f1-score(macro):",metrics_dict['fscore_macro'],"ROC-AUC(macro):",metrics_dict['auc_macro_ovo'],\
             "PR-AUC(macro):",metrics_dict['pr_auc_macro']))

print("{:<20}{:<10,.2%}{:<16}{:<10,.2%}{:<18}{:<10,.2%}{:<17}{:<10,.2%}{:<16}{:<10,.2%}"\
      .format("precision(micro):",metrics_dict['prec_micro'],"recall(micro):",metrics_dict['recall_micro'],\
              "f1-score(micro):",metrics_dict['fscore_micro'],"ROC-AUC(micro):",metrics_dict['auc_micro'],\
             "PR-AUC(micro):",metrics_dict['pr_auc_micro']))

print("{:<20}{:<10,.2%}{:<16}{:<10,.2%}{:<18}{:<10,.2%}{:<17}{:<10,.2%}{:<16}{:<10,.2%}"\
      .format("precision(weight):",metrics_dict['prec_weighted'],"recall(weight):",metrics_dict['recall_weighted'],\
              "f1-score(weight):",metrics_dict['fscore_weighted'],"ROC-AUC(weight):",metrics_dict['auc_weighted_ovo'],\
             "PR-AUC(weight):",metrics_dict['pr_auc_weighted']))

print()
n_classes=th.unique(node_labels_new).shape[0]
label_map={0:"No Product",1:"Rental Only",2:"Home Only",3:"Auto Only",4:"Auto + Rental",5:"Auto + Home"}

report=metrics.classification_report(fin_targets.squeeze(), fin_outputs.argmax(axis=1), output_dict=True)

table = pd.DataFrame(report).transpose().iloc[:n_classes,:]
table["count"]=table["support"].astype(int)
table["roc_auc"]=[roc_auc[i] for i in range(n_classes)]
table["pr_auc"]=[pr_auc[i] for i in range(n_classes)]
table["product_type"]=[label_map[i] for i in range(n_classes)]
table=table[['product_type','count','precision','recall','f1-score','roc_auc','pr_auc']]

total=table['count'].sum()

table.loc[len(table.index)]=["MACRO",total,metrics_dict['prec_macro'],metrics_dict['recall_macro'],metrics_dict['fscore_macro'],\
                        metrics_dict['auc_macro_ovo'],metrics_dict['pr_auc_macro']]

table.loc[len(table.index)]=["MICRO",total,metrics_dict['prec_micro'],metrics_dict['recall_micro'],metrics_dict['fscore_micro'],\
                            metrics_dict['auc_micro'],metrics_dict['pr_auc_micro']]

table.loc[len(table.index)]=["WEIGHT",total,metrics_dict['prec_weighted'],metrics_dict['recall_weighted'],metrics_dict['fscore_weighted'],\
                        metrics_dict['auc_weighted_ovo'],metrics_dict['pr_auc_weighted']]

table.style.format({"count":"{:,}","f1-score":"{:.2%}","precision":"{:.2%}","recall":"{:.2%}","roc_auc":"{:.2%}","pr_auc":"{:.2%}"})

In [ ]:
%%time
print()
print("********  model performance in test set **********  \n ")
print()
    
fin_outputs, fin_targets, losses_tmp=eval_loop_func(model, test_loader, node_labels_new, device, weight, n_classes)
metrics_dict, roc_auc, pr_auc = evaluate(fin_outputs, fin_targets)

print("{:<20}{:<10.2%}".format("accuracy", metrics_dict['acc']))
print()
print("{:<20}{:<10,.2%}{:<16}{:<10,.2%}{:<18}{:<10,.2%}{:<17}{:<10,.2%}{:<16}{:<10,.2%}"\
      .format("precision(macro):",metrics_dict['prec_macro'],"recall(macro):",metrics_dict['recall_macro'],\
              "f1-score(macro):",metrics_dict['fscore_macro'],"ROC-AUC(macro):",metrics_dict['auc_macro_ovo'],\
             "PR-AUC(macro):",metrics_dict['pr_auc_macro']))

print("{:<20}{:<10,.2%}{:<16}{:<10,.2%}{:<18}{:<10,.2%}{:<17}{:<10,.2%}{:<16}{:<10,.2%}"\
      .format("precision(micro):",metrics_dict['prec_micro'],"recall(micro):",metrics_dict['recall_micro'],\
              "f1-score(micro):",metrics_dict['fscore_micro'],"ROC-AUC(micro):",metrics_dict['auc_micro'],\
             "PR-AUC(micro):",metrics_dict['pr_auc_micro']))

print("{:<20}{:<10,.2%}{:<16}{:<10,.2%}{:<18}{:<10,.2%}{:<17}{:<10,.2%}{:<16}{:<10,.2%}"\
      .format("precision(weight):",metrics_dict['prec_weighted'],"recall(weight):",metrics_dict['recall_weighted'],\
              "f1-score(weight):",metrics_dict['fscore_weighted'],"ROC-AUC(weight):",metrics_dict['auc_weighted_ovo'],\
             "PR-AUC(weight):",metrics_dict['pr_auc_weighted']))

print()
n_classes=th.unique(node_labels_new).shape[0]
label_map={0:"No Product",1:"Rental Only",2:"Home Only",3:"Auto Only",4:"Auto + Rental",5:"Auto + Home"}

report=metrics.classification_report(fin_targets.squeeze(), fin_outputs.argmax(axis=1), output_dict=True)

table = pd.DataFrame(report).transpose().iloc[:n_classes,:]
table["count"]=table["support"].astype(int)
table["roc_auc"]=[roc_auc[i] for i in range(n_classes)]
table["pr_auc"]=[pr_auc[i] for i in range(n_classes)]
table["product_type"]=[label_map[i] for i in range(n_classes)]
table=table[['product_type','count','precision','recall','f1-score','roc_auc','pr_auc']]

total=table['count'].sum()

table.loc[len(table.index)]=["MACRO",total,metrics_dict['prec_macro'],metrics_dict['recall_macro'],metrics_dict['fscore_macro'],\
                        metrics_dict['auc_macro_ovo'],metrics_dict['pr_auc_macro']]

table.loc[len(table.index)]=["MICRO",total,metrics_dict['prec_micro'],metrics_dict['recall_micro'],metrics_dict['fscore_micro'],\
                            metrics_dict['auc_micro'],metrics_dict['pr_auc_micro']]

table.loc[len(table.index)]=["WEIGHT",total,metrics_dict['prec_weighted'],metrics_dict['recall_weighted'],metrics_dict['fscore_weighted'],\
                        metrics_dict['auc_weighted_ovo'],metrics_dict['pr_auc_weighted']]

table.style.format({"count":"{:,}","f1-score":"{:.2%}","precision":"{:.2%}","recall":"{:.2%}","roc_auc":"{:.2%}","pr_auc":"{:.2%}"})

In [ ]:
# %%time
# print("Classification Report for Training Dataset: \n")
# print()
# precision_macro, recall_macro, fscore_macro, _ = score(train_true, train_pred, average='macro')
# precision_micro, recall_micro, fscore_micro, _ = score(train_true, train_pred, average='micro')
# precision_weighted, recall_weighted, fscore_weighted, _ = score(train_true, train_pred, average='weighted')
# print("{:<20}{:<10,.2%}{:<18}{:<10,.2%}{:<20}{:<10,.2%}".format("precision(macro):",precision_macro,"recall(macro):",recall_macro,"f1-score(macro):",fscore_macro))
# print("{:<20}{:<10,.2%}{:<18}{:<10,.2%}{:<20}{:<10,.2%}".format("precision(micro):",precision_micro,"recall(micro):",recall_micro,"f1-score(micro):",fscore_micro))
# print("{:<20}{:<10,.2%}{:<18}{:<10,.2%}{:<20}{:<10,.2%}".format("precision(weight):",precision_weighted,"recall(weight):",recall_weighted,"f1-score(weight):",fscore_weighted))
# print()
# print(metrics.classification_report(train_true, train_pred, digits=3))

In [ ]:
#  '000': 0,  17,906,619
#  '001': 1,     222,153
#  '010': 2,     206,359
#  '011': 3,       2,522
#  '100': 4,   2,123,313
#  '101': 5,   1,257,366
#  '110': 6,   2,790,185
#  '111': 7,      53,393

#### sparsity rate of embedding vector

In [ ]:
H=np.array(H)
non_zero=np.count_nonzero(H)
total_val=np.product(H.shape)
sparsity=(total_val-non_zero)/total_val
density=non_zero/total_val
print("sparsity rate is {:.2%}".format(sparsity))
print("density rate is {:.2%}".format(density))
print("embedding vector shape is {}".format(H.shape))

In [ ]:
IDX_train=np.array(IDX)
H_train=np.array(H)
mask_train=np.array(['train']*len(IDX_train))

In [ ]:
IDX_val=[]
H_val=[]
for input_nodes_raw, seeds_raw, blocks in tqdm(val_loader, position=0, leave=True):
    blocks = [blk.to(device) for blk in blocks]
    seeds=seeds_raw.to(device)
    input_nodes={}
    input_nodes["usaanr"]=input_nodes_raw
    input_nodes={k : e.to(device) for k, e in input_nodes.items()}
    model.eval()
    logits,h = model(input_nodes,blocks)
    IDX_val.extend(seeds.detach().cpu().numpy().tolist())
    H_val.extend(h.detach().cpu().numpy().tolist())
    
IDX_val=np.array(IDX_val)
H_val=np.array(H_val)
mask_val=np.array(['val']*len(IDX_val))

In [ ]:
IDX_test=[]
H_test=[]
for input_nodes_raw, seeds_raw, blocks in tqdm(test_loader, position=0, leave=True):
    blocks = [blk.to(device) for blk in blocks]
    seeds=seeds_raw.to(device)
    input_nodes={}
    input_nodes["usaanr"]=input_nodes_raw
    input_nodes={k : e.to(device) for k, e in input_nodes.items()}
    model.eval()
    logits,h = model(input_nodes,blocks)
    IDX_test.extend(seeds.detach().cpu().numpy().tolist())
    H_test.extend(h.detach().cpu().numpy().tolist())
    
IDX_test=np.array(IDX_test)
H_test=np.array(H_test)
mask_test=np.array(['test']*len(IDX_test))

In [ ]:
IDX=np.concatenate((IDX_train,IDX_val, IDX_test))
H=np.concatenate((H_train,H_val, H_test))
mask=np.concatenate((mask_train,mask_val, mask_test))

_idx=IDX.argsort()  #### sort the node id from 0 to max_num, so that it can be matched with LABEL
embedding_vector=H[_idx]
mask=mask[_idx]

print("{:<30}{}".format("shape of embedding vector",embedding_vector.shape))

In [ ]:
# %%time
# savez_compressed("embedding_vector.npz",embedding_vector)
# embedding_vector=load("embedding_vector.npz")['arr_0']

In [ ]:
# %%time
# embedding_vector=load("embedding_vector.npz")['arr_0']

#### visualization of embedding vectors for different categories of USAA Members

In [ ]:
DF=pd.DataFrame({"IDX":_idx.tolist(), "MASK":mask.tolist(), "Product":LABEL.squeeze().tolist()})
prod_map={0:"No_Product",1:"Rental_only",2:"Home_only",3:"Auto_only",4:"Auto+Rental",5:"Auto+Home"}
DF['Product'] = list(map(prod_map.get, DF['Product']))

### the caterogry of "Home+Rental" is too rare, only 2767 in total. So remove it.
# inx=np.where(DF["Product"].values!="Home+Rental")[0]
# DF=DF[DF["Product"]!="Home+Rental"]
# embedding_vector=embedding_vector[inx]
# DF["IDX"]=np.arange(DF.shape[0])  ## Reindex the DF since the caterogry of "Home+Rental" has been removed

N=1000

# sample_df=DF.groupby(['Product'], group_keys=False).apply(lambda x: x.sample(n=N))
# sample_id=sample_df["IDX"].values
# embedding_sample=embedding_vector[sample_id]
# prod_sample=DF['Product'].values[sample_id]

train_df=DF[DF["MASK"]=="train"]
sample_df_train=train_df.groupby(['Product'], group_keys=False).apply(lambda x: x.sample(n=N,random_state=101))
sample_id_train=sample_df_train["IDX"].values
prod_sample_train=DF['Product'].values[sample_id_train]
embedding_sample_train=embedding_vector[sample_id_train]

val_df=DF[DF["MASK"]=="val"]
sample_df_val=val_df.groupby(['Product'], group_keys=False).apply(lambda x: x.sample(n=N,random_state=101))
sample_id_val=sample_df_val["IDX"].values
prod_sample_val=DF['Product'].values[sample_id_val]
embedding_sample_val=embedding_vector[sample_id_val]

test_df=DF[DF["MASK"]=="test"]
sample_df_test=test_df.groupby(['Product'], group_keys=False).apply(lambda x: x.sample(n=N,random_state=101))
sample_id_test=sample_df_test["IDX"].values
prod_sample_test=DF['Product'].values[sample_id_test]
embedding_sample_test=embedding_vector[sample_id_test]

In [ ]:
%%time
print("t-SNE dimension reduction for training embedding vector:")
print()
train_embedding = tsne_func.fit(embedding_sample_train)

savez_compressed("RGCN_train_embedding_tsne.npz",train_embedding)
train_embedding=load("RGCN_train_embedding_tsne.npz")['arr_0']


In [ ]:
%%time
print("t-SNE dimension reduction for validation embedding:")
print()
val_embedding = tsne_func.fit(embedding_sample_val)

savez_compressed("RGCN_val_embedding_tsne.npz",val_embedding)
val_embedding=load("RGCN_val_embedding_tsne.npz")['arr_0']

In [ ]:
%%time
print("t-SNE dimension reduction for test embedding:")
print()
test_embedding = tsne_func.fit(embedding_sample_test)

savez_compressed("RGCN_test_embedding_tsne.npz",test_embedding)
test_embedding=load("RGCN_test_embedding_tsne.npz")['arr_0']

In [ ]:
a,b=np.unique(prod_sample_train,return_counts=True)
pd.DataFrame({"product":a,"sampled_#":b}).style.format({'sampled_#':'{:,}'})

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 7)
ax=sns.scatterplot(train_embedding[:,0], train_embedding[:,1], hue=prod_sample_train.squeeze(), legend='full', palette=palette)
ax.set_title("Embedding Vectors t-SNE(sample=1000) \n Training Set ",fontsize=15)

In [ ]:
a,b=np.unique(prod_sample_val,return_counts=True)
pd.DataFrame({"product":a,"sampled_#":b}).style.format({'sampled_#':'{:,}'})

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 7)
ax=sns.scatterplot(val_embedding[:,0], val_embedding[:,1], hue=prod_sample_val.squeeze(), legend='full', palette=palette)
ax.set_title("Embedding Vectors t-SNE (sample=1000) \n Validation Set ",fontsize=15)

In [ ]:
a,b=np.unique(prod_sample_test,return_counts=True)
pd.DataFrame({"product":a,"sampled_#":b}).style.format({'sampled_#':'{:,}'})

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 7)
ax=sns.scatterplot(test_embedding[:,0], test_embedding[:,1], hue=prod_sample_test.squeeze(), legend='full', palette=palette)
ax.set_title("Embedding Vectors t-SNE (sample=1000) \n Test Set ",fontsize=15)